<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=26a07e8fa5980a45092f516de873ffe8d6e3aaa7d8f5468a2ea1255c11e5cfa1
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-18 15:17:18
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: 29.98 K (0.21%)
Current PnL: -16.74 L (-11.26%)
CY Booked + Current PnL: -5.30 L (-3.57%)
-------------------
Total profit:  2.12 L
Total loss:  -18.86 L
-------------------
Total Booked + Current PnL: 21.68 L (17.81%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.24 C
Est FTT PnL: 81.28 L (56.89%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.94%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.77,65.0,H-MC,3.57,196290.0,31620.0,11719.0,1.17,19.20,5.97,26.31,89.0,2.70,1.44,28.72,X40N,NTT,AC
76,TTKPRESTIG,770.00,104.48,57.0,M-SC,3.97,88071.0,-12706.0,12797.0,-0.66,-12.61,14.53,0.09,245.0,-0.99,0.64,15.71,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,2.00,61.0,H-MC,2.51,213855.0,22113.0,15654.0,0.98,11.53,7.32,19.69,99.0,1.41,1.56,19.11,XY25,NTT,AC
49,NATIONALUM,247.44,-43.66,65.0,H-MC,6.83,113203.0,12699.0,17943.0,0.16,12.64,15.85,30.49,79.0,0.71,0.83,49.13,MH,ATH,METALS
56,RELIANCE,1533.00,-11.79,60.0,H-LC,5.19,220740.0,10374.0,18410.0,0.08,4.93,8.34,13.68,37.0,0.56,1.61,22.27,XY25,NTT,REFINERIES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,IEX,219.00,-29.46,56.0,H-SC,17.06,201501.0,1547.0,95673.0,0.03,0.77,47.48,48.63,115.0,0.02,1.47,12.42,XR,NTT,MISC
34,ICICIPRULI,790.00,-20.54,46.0,H-MC,2.22,136753.0,965.0,41792.0,0.83,0.71,30.56,31.48,107.0,0.02,1.00,12.91,X40,ATH,INSURANCE
29,HAVELLS,2069.16,0.34,59.0,H-MC,1.88,248648.0,917.0,74147.0,0.02,0.37,29.82,30.30,92.0,0.01,1.82,13.71,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-30.81,52.0,H-MC,4.21,200175.0,-645.0,69421.0,-0.29,-0.32,34.68,34.25,80.0,-0.01,1.46,4.86,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-15.98,43.0,H-LC,6.33,302540.0,-13302.0,125978.0,0.67,-4.21,41.64,35.67,5.0,-0.11,2.21,9.06,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-13.58,44.0,H-LC,7.80,218126.0,-33642.0,86858.0,-0.60,-13.36,39.82,21.13,27.0,-0.39,1.60,17.33,X40,ATH,PAINTS
43,JIOFIN,387.00,-0.02,51.0,H-LC,13.31,271631.0,4266.0,61198.0,0.13,1.60,22.53,24.48,48.0,0.07,1.99,57.44,XY24,BTT,FINANCE
41,ITC,452.00,-37.50,52.0,H-LC,2.51,201615.0,1477.0,19416.0,0.73,0.74,9.63,10.44,4.0,0.08,1.47,6.34,X40,NTT,FMCG
79,UNITDSPR,1644.00,-12.94,52.0,H-LC,7.80,228330.0,-5814.0,54434.0,-0.72,-2.48,23.84,20.77,86.0,-0.11,1.67,4.26,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,0.29,71.0,H-LC,1.28,248895.0,2744.0,76461.0,1.99,1.11,30.72,32.18,16.0,0.04,1.82,37.56,X200,ATH,IT
41,ITC,452.0,-37.50,52.0,H-LC,2.51,201615.0,1477.0,19416.0,0.73,0.74,9.63,10.44,4.0,0.08,1.47,6.34,X40,NTT,FMCG
50,NESTLEIND,1377.0,-7.97,58.0,H-LC,3.86,283023.0,17597.0,39199.0,0.43,6.63,13.85,21.40,11.0,0.45,2.07,13.71,XY25,NTT,FMCG
56,RELIANCE,1533.0,-11.79,60.0,H-LC,5.19,220740.0,10374.0,18410.0,0.08,4.93,8.34,13.68,37.0,0.56,1.61,22.27,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.09,56.0,H-LC,5.81,216186.0,11686.0,29726.0,1.18,5.71,13.75,20.25,10.0,0.39,1.58,14.95,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6927.78,62.0,L-SC,5.25,77128.0,-16682.0,93302.0,4.10,-17.78,120.97,81.67,269.0,-0.18,0.56,49.36,XR,NTT,CERAMICS
60,SAMMAANCAP,326.00,-172.73,66.0,M-SC,2.68,83604.0,-18606.0,111996.0,3.16,-18.20,133.96,91.37,208.0,-0.17,0.61,35.72,XY25,NTT,FINANCE
63,SHALBY,327.00,1092.98,68.0,M-SC,3.62,162301.0,-18842.0,64304.0,2.79,-10.40,39.62,25.10,235.0,-0.29,1.19,30.07,XY24,NTT,HEALTHCARE
15,BSOFT,836.99,-11.70,58.0,H-SC,15.01,99106.0,-41547.0,114329.0,2.01,-29.54,115.36,51.74,131.0,-0.36,0.73,13.02,XR,ATH,IT
47,LTIM,7230.20,0.29,71.0,H-LC,1.28,248895.0,2744.0,76461.0,1.99,1.11,30.72,32.18,16.0,0.04,1.82,37.56,X200,ATH,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GREENPANEL,537.00,229.46,60.0,M-SC,2.50,147010.0,-36068.0,115050.0,-2.81,-19.70,78.26,43.14,230.0,-0.31,1.08,38.28,XY24,NTT,MISC
54,RAJOOENG,143.33,-37.02,36.0,H-SC,11.64,95440.0,-7060.0,47892.0,-2.61,-6.89,50.18,39.83,114.0,-0.15,0.70,0.00,AR,ATH,MISC
36,INDIAMART,4850.92,-51.65,44.0,H-SC,11.99,135192.0,11856.0,121903.0,-1.81,9.61,90.17,108.45,119.0,0.10,0.99,34.11,AR,ATH,MISC
57,REPCOHOME,880.00,-59.76,52.0,H-SC,6.43,180521.0,-55222.0,244516.0,-1.79,-23.42,135.45,80.30,134.0,-0.23,1.32,22.20,XY24,NTT,FINANCE
20,COALINDIA,497.97,8.96,59.0,L-LC,2.36,145410.0,-5050.0,38839.0,-1.69,-3.36,26.71,22.46,167.0,-0.13,1.06,14.11,XY25,ATH,MINING


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,104.48,57.0,M-SC,3.97,88071.0,-12706.0,12797.0,-0.66,-12.61,14.53,0.09,245.0,-0.99,0.64,15.71,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,93.70,53.0,H-SC,9.51,122661.0,-19818.0,39337.0,-0.70,-13.91,32.07,13.70,163.0,-0.50,0.90,44.77,OX40N,NTT,BANKS
44,KANSAINER,340.0,-66.87,65.0,H-SC,1.38,228150.0,-41517.0,77845.0,1.61,-15.40,34.12,13.47,138.0,-0.53,1.67,15.40,XY24,NTT,PAINTS
18,CERA,9475.0,-19.81,47.0,H-SC,2.10,146050.0,-29853.0,71871.0,-0.38,-16.97,49.21,23.89,149.0,-0.42,1.07,26.13,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-19.51,55.0,H-MC,7.65,108965.0,-22870.0,65096.0,0.92,-17.35,59.74,32.03,98.0,-0.35,0.80,22.05,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,60.0,H-LC,12.33,298600.0,-47356.0,122217.0,0.12,-13.69,40.93,21.64,1.0,-0.39,2.18,5.78,X40,ATH,IT
39,INFY,2275.00,-15.65,58.0,H-LC,8.42,327999.0,15111.0,156587.0,1.15,4.83,47.74,54.87,3.0,0.10,2.40,11.80,X40,BTT,IT
41,ITC,452.00,-37.50,52.0,H-LC,2.51,201615.0,1477.0,19416.0,0.73,0.74,9.63,10.44,4.0,0.08,1.47,6.34,X40,NTT,FMCG
82,VBL,671.64,-15.98,43.0,H-LC,6.33,302540.0,-13302.0,125978.0,0.67,-4.21,41.64,35.67,5.0,-0.11,2.21,9.06,X40N,ATH,FMCG
1,ABB,7934.00,-37.53,68.0,H-LC,10.33,260952.0,-667.0,119881.0,0.94,-0.26,45.94,45.57,7.0,-0.01,1.91,11.04,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,45.15,65.0,M-MC,6.97,228647.0,3685.0,163963.0,0.02,1.64,71.71,74.53,192.0,0.02,1.67,34.42,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.77,65.0,H-MC,3.57,196290.0,31620.0,11719.0,1.17,19.20,5.97,26.31,89.0,2.70,1.44,28.72,X40N,NTT,AC
83,VOLTAS,1530.00,2.00,61.0,H-MC,2.51,213855.0,22113.0,15654.0,0.98,11.53,7.32,19.69,99.0,1.41,1.56,19.11,XY25,NTT,AC
17,CAMS,5211.76,0.38,66.0,H-SC,3.81,114036.0,12032.0,31896.0,1.47,11.80,27.97,43.06,122.0,0.38,0.83,30.93,X40N,ATH,MISC
29,HAVELLS,2069.16,0.34,59.0,H-MC,1.88,248648.0,917.0,74147.0,0.02,0.37,29.82,30.30,92.0,0.01,1.82,13.71,X40,ATH,ELECTRICAL


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-43.66,65.0,H-MC,6.83,113203.0,12699.0,17943.0,0.16,12.64,15.85,30.49,79.0,0.71,0.83,49.13,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.54,52.0,M-SC,0.48,100500.0,9002.0,69747.0,-0.12,9.84,69.40,86.07,223.0,0.13,0.74,45.94,XR,NTT,DURABLES
36,INDIAMART,4850.92,-51.65,44.0,H-SC,11.99,135192.0,11856.0,121903.0,-1.81,9.61,90.17,108.45,119.0,0.10,0.99,34.11,AR,ATH,MISC
85,WIPRO,420.00,-9.26,60.0,M-LC,5.63,159253.0,8308.0,101142.0,1.07,5.50,63.51,72.51,53.0,0.08,1.17,11.86,XR,NTT,IT
11,BANKINDIA,190.00,-29.19,66.0,H-MC,5.97,184006.0,4198.0,108214.0,1.80,2.33,58.81,62.52,88.0,0.04,1.35,37.22,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6927.78,62.0,L-SC,5.25,77128.0,-16682.0,93302.0,4.10,-17.78,120.97,81.67,269.0,-0.18,0.56,49.36,XR,NTT,CERAMICS
60,SAMMAANCAP,326.00,-172.73,66.0,M-SC,2.68,83604.0,-18606.0,111996.0,3.16,-18.20,133.96,91.37,208.0,-0.17,0.61,35.72,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-43.66,51.0,L-SC,42.26,84288.0,-29261.0,69310.0,-0.97,-25.77,82.23,35.27,268.0,-0.42,0.62,106.30,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.17,48.0,H-SC,12.74,89234.0,-11729.0,106260.0,-0.89,-11.62,119.08,93.63,148.0,-0.11,0.65,32.91,SR,ATH,CHEMICALS
48,MASFIN,398.61,-17.34,51.0,H-SC,12.98,94140.0,-3840.0,25446.0,-0.81,-3.92,27.03,22.05,152.0,-0.15,0.69,36.84,XR,ATH,FINANCE


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.0,3894.59,64.0,M-SC,5.82,124584.0,-22784.0,74601.0,0.81,-15.46,59.88,35.16,236.0,-0.31,0.91,27.25,XY24,NTT,AUTO
60,SAMMAANCAP,326.0,-172.73,66.0,M-SC,2.68,83604.0,-18606.0,111996.0,3.16,-18.20,133.96,91.37,208.0,-0.17,0.61,35.72,XY25,NTT,FINANCE
63,SHALBY,327.0,1092.98,68.0,M-SC,3.62,162301.0,-18842.0,64304.0,2.79,-10.40,39.62,25.10,235.0,-0.29,1.19,30.07,XY24,NTT,HEALTHCARE
21,COFFEEDAY,80.0,-43.66,51.0,L-SC,42.26,84288.0,-29261.0,69310.0,-0.97,-25.77,82.23,35.27,268.0,-0.42,0.62,106.30,XR,NTT,HOTELS
80,VAIBHAVGBL,521.0,65.84,71.0,H-SC,7.26,143713.0,-39062.0,150137.0,0.64,-21.37,104.47,60.77,125.0,-0.26,1.05,28.72,XR,NTT,JEWELLERY


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.10
1,25,43.90
2,50,74.46


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.54
LC    32.15
MC    23.31
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.25
X40      15.43
X40N     11.99
XR       11.81
AR        8.76
XY25      8.26
OX40N     8.22
X200      1.82
MH        1.72
X5K       1.45
SR        1.29
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.30
H-LC    25.44
H-MC    20.34
M-SC    13.66
M-LC     5.65
M-MC     2.66
L-SC     1.58
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.70,-12.34,73.60
FMCG,11.90,-3.94,40.31
FINANCE,9.74,-15.66,60.96
MISC,7.47,-12.56,75.16
BANKS,6.32,-15.67,76.53
PAINTS,5.88,-14.93,32.06
ELECTRICAL,5.77,-4.47,62.06
AC,3.72,4.81,11.13
INSURANCE,3.44,-6.88,43.54


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2846118.0,22
AR,1299021.0,9
XR,1298114.0,14
X40,739439.0,10
X40N,566469.0,9
OX40N,523738.0,10
XY25,333443.0,6
SR,244077.0,2
X5K,119930.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3407085.0,29
M-SC,1378652.0,17
H-LC,1213805.0,14
H-MC,1090223.0,15
M-LC,379582.0,4
M-MC,313332.0,2
L-SC,245646.0,3
L-MC,60659.0,1
L-LC,38839.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1027270.0      6
M-SC       XY24       796535.0      7
H-SC       AR         786147.0      5
           XR         750833.0      7
H-MC       XY24       489273.0      4
H-LC       X40        455760.0      5
           AR         352639.0      2
M-MC       XY24       313332.0      2
H-SC       X40N       284741.0      4
           OX40N      250947.0      4
           SR         244077.0      2
H-MC       X40        213108.0      4
H-LC       X40N       210138.0      3
L-SC       XR         162612.0      2
M-SC       AR         160235.0      2
M-LC       XY24       158510.0      2
H-MC       XY25       124999.0      2
M-SC       OX40N      124661.0      4
M-LC       X5K        119930.0      1
M-SC       XR         114654.0      2
           XY25       111996.0      1
H-MC       XR         108214.0      1
M-LC       XR         101142.0      1
L-SC       OX40N       83034.0      1
H-LC       X200        76461.0      1
H-MC       X40N        71590.0      2
M-SC       X40         70571.0      1
H-MC       OX40N       65096.0      1
H-SC       MH          63070.0      1
H-LC       XY24        61198.0      1
L-MC       XR          60659.0      1
H-LC       XY25        57609.0      2
L-LC       XY25        38839.0      1
H-MC       MH          17943.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
